In [1]:
# !pip install datasets transformers
# !pip install sentencepiece
# !pip install -U scikit-learn

In [2]:
import os
import pandas as pd
import numpy as np
#from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
import shutil
import time
import gc
import random
import math
import torch
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import TrainingArguments, Trainer , DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
os.environ["WANDB_DISABLED"] = "true"

In [3]:
df = pd.read_csv('../../uspatent/train.csv')
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [4]:
df_context = pd.read_csv('../../uspatent/titles.csv')
df_context

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0
...,...,...,...,...,...,...,...
260471,Y10T483/1864,including tool pot or adapter,Y,10.0,T,483.0,1864.0
260472,Y10T483/1873,Indexing matrix,Y,10.0,T,483.0,1873.0
260473,Y10T483/1882,Rotary disc,Y,10.0,T,483.0,1882.0
260474,Y10T483/1891,Chain or belt,Y,10.0,T,483.0,1891.0


In [5]:
df = df.merge(df_context, how='left', left_on='context', right_on='code')
df

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,B44,DECORATIVE ARTS,B,44.0,NaN,NaN,NaN
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,B44,DECORATIVE ARTS,B,44.0,NaN,NaN,NaN
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,B44,DECORATIVE ARTS,B,44.0,NaN,NaN,NaN
36471,756ec035e694722b,wood article,wooden material,B44,0.75,B44,DECORATIVE ARTS,B,44.0,NaN,NaN,NaN


In [6]:
df = df[['id', 'anchor', 'target', 'context', 'title', 'score']]
df

,id,anchor,target,context,title,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.00
...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,DECORATIVE ARTS,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,DECORATIVE ARTS,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,DECORATIVE ARTS,0.50
36471,756ec035e694722b,wood article,wooden material,B44,DECORATIVE ARTS,0.75


## 从当前目录下面的东西弄进来

In [7]:
from transformers import AutoTokenizer
tonkenizer = AutoTokenizer.from_pretrained('./deberta-v3-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
df['input'] = df['target'] + tonkenizer.sep_token + df['title'].apply(str.lower)

C:\Users\Administrator\anaconda3\envs\photo_fix\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
df

,id,anchor,target,context,title,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50,abatement of pollution[SEP]furniture; domestic...
1,7b9652b17b68b7a4,abatement,act of abating,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.75,act of abating[SEP]furniture; domestic article...
2,36d72442aefd8232,abatement,active catalyst,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.25,active catalyst[SEP]furniture; domestic articl...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.50,eliminating process[SEP]furniture; domestic ar...
4,54c1e3b9184cb5b6,abatement,forest region,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,0.00,forest region[SEP]furniture; domestic articles...
...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,DECORATIVE ARTS,1.00,wooden article[SEP]decorative arts
36469,42d9e032d1cd3242,wood article,wooden box,B44,DECORATIVE ARTS,0.50,wooden box[SEP]decorative arts
36470,208654ccb9e14fa3,wood article,wooden handle,B44,DECORATIVE ARTS,0.50,wooden handle[SEP]decorative arts
36471,756ec035e694722b,wood article,wooden material,B44,DECORATIVE ARTS,0.75,wooden material[SEP]decorative arts


In [10]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
train_df

,id,anchor,target,context,title,score,input
1715,4d0c110393aa7804,antigen composition,antigen immunoassay,G01,MEASURING; TESTING,0.50,antigen immunoassay[SEP]measuring; testing
26508,442fda7b257e7501,pulping apparatus,image forming apparatus,D06,TREATMENT OF TEXTILES OR THE LIKE; LAUNDERING;...,0.00,image forming apparatus[SEP]treatment of texti...
15967,7071bc0449df0e69,hydrocarbyl substituted succinic,substituted succinic anhydride,C10,"PETROLEUM, GAS OR COKE INDUSTRIES; TECHNICAL G...",0.25,"substituted succinic anhydride[SEP]petroleum, ..."
17306,c14c239f5ff44628,intermediate connection,intermediate eye color,B41,PRINTING; LINING MACHINES; TYPEWRITERS; STAMPS,0.00,intermediate eye color[SEP]printing; lining ma...
1741,d9de84df01533861,antigen composition,poetic composition,G01,MEASURING; TESTING,0.00,poetic composition[SEP]measuring; testing
...,...,...,...,...,...,...,...
16850,f2249a1af855005e,inner peripheral,inner peripheral side,G11,INFORMATION STORAGE,0.75,inner peripheral side[SEP]information storage
6265,a47f92ba501b2d69,committee,approval area,G07,CHECKING-DEVICES,0.00,approval area[SEP]checking-devices
11284,cc064489bbf655c1,embedding groove,elongated groove,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,0.50,elongated groove[SEP]combustion apparatus; com...
860,66583335020bcf02,aesthetic effects,aesthetic values,A63,SPORTS; GAMES; AMUSEMENTS,0.50,aesthetic values[SEP]sports; games; amusements


In [12]:
train_df.shape

(29178, 7)

In [13]:
val_df.shape

(7295, 7)

In [65]:
import torch
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, df):
        self.input = df['input'].values.astype(str)
        self.anchor = df['anchor'].values.astype(str)
        self.label = df['score'].values
    
    def __getitem__(self, item):
        inputs = self.input[item]
        anchor = self.anchor[item]
        label = self.label[item]
        
        model_inputs = tonkenizer(inputs, anchor,
                                max_length=100,
                                padding='max_length',
                                truncation=True)
        # 把model_inputs作为字典返回出去，你看下面的lable是不是就是一个字典
        return {**model_inputs,
               'label':torch.as_tensor(label, dtype=torch.float)}
    
    def __len__(self):
        return len(self.input)


In [15]:
tonkenizer('I love China', 'I love Shanghai')

{'input_ids': [1, 273, 472, 1208, 2, 273, 472, 9458, 2], 'token_type_ids': [0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tonkenizer('I love China I love Shanghai')['token_type_ids']

[0, 0, 0, 0, 0, 0, 0, 0]

### stsb：语义-文本相似性基准（Cer等人，2017）是从新闻标题、视频和图像标题以及自然语言推理数据中提取的句子对的集合。每一对都是人类注释，相似性分数从0到5

### glue: 包括了上面的stsb, 下面打印了例子你认真看看呗


#### 就是说glue包括了sst2 mrpc stsb cola
#### datasets.load_metric('glue', 'sst2')
#### datasets.load_metric('glue', 'mrpc')
#### datasets.load_metric('glue', 'stsb')
#### datasets.load_metric('glue', 'cola')

### 下面这段代码我测试过了，不用也行，是我在kaggleinference的时候，那时候不能联网，但是这个有需要联网，就冲突了

In [4]:
# from datasets import load_metric
# metric = load_metric('glue', 'stsb')
# metric

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./deberta-v3-small', num_labels=1)

Some weights of the model checkpoint at ./deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

In [19]:
# 

In [20]:
from transformers import TrainingArguments, Trainer

### pearson:皮尔逊相关系

### TrainingArguments 中model_test是输出文件夹，其他的都差不多


### 其他的在这里： https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/trainer#transformers.TrainingArguments

In [21]:
# evaluation_strategy: 
#     训练期间采用的评估策略。可能的值为：
#     "no": 训练期间不进行评估。
#     "steps"：评估完成（并记录）每eval_steps.
#     "epoch": 在每个 epoch 结束时进行评估。

# save_strategy（str或IntervalStrategy，可选，默认为"steps"）——训练期间采用的检查点保存策略。可能的值为：
#    "no"：训练期间不进行保存。
#    "epoch"：保存在每个 epoch 结束时完成。
#    "steps": 保存每save_steps.
        


In [22]:
metric_name = 'pearson'
batch_size = 64
args = TrainingArguments(
       'model_test',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate = 2e-5,
    # 训练时，每个GPUd的大小
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    num_train_epochs=5,
    weight_decay=0.01,
    # 是否在训练结束时加载训练期间找到的最佳模型
    load_best_model_at_end=True,
    # 上面是加载最佳模型，这个是告诉用什么评判最佳模型
    metric_for_best_model=metric_name,
    #  如果设置具体数值，将限制检查点的总数。删除中的旧检查点，即只保留一个
    save_total_limit=1
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [23]:
# def compute_metrics(eval_pred):
#     pred, label = eval_pred
#     return metric.compute(predictions=pred, references=lable)

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.reshape(len(predictions))
    return {
        'pearson': np.corrcoef(predictions, labels)[0][1]
    }


In [25]:
train_dataset = TrainDataset(train_df)
val_dataset = TrainDataset(val_df)

trainer = Trainer(model,
                  args,
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset,
                  tokenizer=tonkenizer,
                  compute_metrics=compute_metrics
                 )

In [26]:
trainer.train()

***** Running training *****
  Num examples = 29178
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 2280


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030580,0.785246
2,0.045700,0.025357,0.817422
3,0.025500,0.026608,0.824801
4,0.020700,0.025569,0.829967
5,0.017800,0.023951,0.831013


***** Running Evaluation *****
  Num examples = 7295
  Batch size = 128
Saving model checkpoint to model_test\checkpoint-456
Configuration saved in model_test\checkpoint-456\config.json
Model weights saved in model_test\checkpoint-456\pytorch_model.bin
tokenizer config file saved in model_test\checkpoint-456\tokenizer_config.json
Special tokens file saved in model_test\checkpoint-456\special_tokens_map.json
added tokens file saved in model_test\checkpoint-456\added_tokens.json
***** Running Evaluation *****
  Num examples = 7295
  Batch size = 128
Saving model checkpoint to model_test\checkpoint-912
Configuration saved in model_test\checkpoint-912\config.json
Model weights saved in model_test\checkpoint-912\pytorch_model.bin
tokenizer config file saved in model_test\checkpoint-912\tokenizer_config.json
Special tokens file saved in model_test\checkpoint-912\special_tokens_map.json
added tokens file saved in model_test\checkpoint-912\added_tokens.json
Deleting older checkpoint [model_tes

TrainOutput(global_step=2280, training_loss=0.02612550927881609, metrics={'train_runtime': 963.0534, 'train_samples_per_second': 151.487, 'train_steps_per_second': 2.367, 'total_flos': 3774611824110000.0, 'train_loss': 0.02612550927881609, 'epoch': 5.0})

In [27]:
# CUDA_VISBLE_DEVICES=0,1,2,3 python run_train.py

In [ ]:
# 保存模型，给inference用

In [64]:
torch.save({'model': model.state_dict()},'debert-v3-small.pth')

### np.clip函数是一个截取函数，用于截取数组中小于或者大于某值的部分，并使得被截取部分等于固定值。
该函数的作用是将数组a中的所有数限定到范围a_min和a_max中。
部分参数解释：
a：输入矩阵；


a_min：被限定的最小值，所有比a_min小的数都会强制变为a_min；


a_max：被限定的最大值，所有比a_max大的数都会强制变为a_max；


out：可以指定输出矩阵的对象，shape与a相同


In [2]:
# outputs = np.clip(outputs, 0, 1)